In [ ]:
def parse_parenthesis(expr):
    def _helper(iter, pre_type):
        items = []
        for item in iter:
            if item == '(':
                result = _helper(iter, '(')
                items.append(result)
            elif item == '[':
                result = _helper(iter, '[')
                items.append(result)
            elif item == '{':
                result = _helper(iter, '{')
                items.append(result)
            elif pre_type == '(' and item == ')':
                return items
            elif pre_type == '[' and item == ']':
                return items
            elif pre_type == '{' and item == '}':
                return items
            else:
                items.append(item)
        return items
    return _helper(iter(expr), None)

def concate(items):
    content = ''
    replacement = []
    for item in items:
        if type(item) is str:
            content += item
        if type(item) is list:
            replacement.append(content.strip())
            replacement.append(concate(item))
            content = ''
    replacement.append(content.strip())
    return replacement

def remove_len(items):
    temp = []
    for item in items:
        if type(item) is str:
            if item is not "":
                temp.append(item)
        if type(item) is list:
            l = remove_len(item)
            if len(l) >= 1:
                temp.append(l)
    return temp

def parse_tree_structure(items):
    level_numbranches_mapping = {}
    recursions = []
    
    def scan_exhaustive(level, items):
        num_branches = 0
        for item in items:
            if type(item) == list:
                num_branches += 1
                scan_exhaustive(level + 1, item)
        recursions.append((level, num_branches))
        return level, num_branches
    scan_exhaustive(0, items)
    for recursion in recursions:
        level_numbranches_mapping[recursion[0]] = level_numbranches_mapping.get(recursion[0], 0) + recursion[1]
    level_numbranches_mapping = sorted(level_numbranches_mapping.items(), key=lambda x: x[0])
    return level_numbranches_mapping

In [12]:
def flatten(text):
    _list = []
    def helper(items):
        for item in items:
            if type(item) is str:
                _list.append(item)
            if type(item) is list:
                helper(item)
    helper(text)
    return _list

In [2]:
class HierarchicalStructure():
    def __init__(self, text):
        self.text = text
    def analyze_text_hierarchical_structure(self):
        self.analyzed_structure = remove_len(concate(parse_parenthesis(self.text)))
        return self.analyzed_structure
    def analyze_tree_structure(self):
        self.tree_structure = parse_tree_structure(self.analyze_text_hierarchical_structure())
        return self.tree_structure, self.analyzed_structure

In [3]:
def apply_analyze_structure(text):
    opt = HierarchicalStructure(text)
    tree_structure, text_structure = opt.analyze_tree_structure()
    return (tree_structure, text_structure)

In [4]:
def compare_2tree_structure(tree1, tree2):
    if len(tree1) == len(tree2):
        for idx, (level, num_branches) in enumerate(tree1):
            if tree2[idx][0] != level or tree2[idx][1] != num_branches:
                return 0
        return 1
    return 0

In [5]:
def analyze_parenthese_structure(raw_error_logs):
    import pandas as pd
    series = pd.Series(raw_error_logs)
    structures = series.apply(apply_analyze_structure).tolist()
    docid_structure_mapping = dict(zip(range(len(series)), structures))
    docid_errorlog_mapping = dict(zip(range(len(series)), series))
    parenthese_structure_similarity = np.zeros((len(series), len(series)))
    for i in range(len(series)):
        for j in range(i, len(series)):
            tree1 = docid_structure_mapping[i][0]
            tree2 = docid_structure_mapping[j][0]
            score = compare_2tree_structure(tree1, tree2)
            parenthese_structure_similarity[i][j] = score
            parenthese_structure_similarity[j][i] = score
    return parenthese_structure_similarity